# Setup

Import the libraries

In [1]:
''' original from Ioanna '''

# import importlib
# #import PyUncertainNumber.UP.local_optimisation as up


# import PyUncertainNumber.UP.main as main
# #importlib.reload(up)
# importlib.reload(main)

# from PyUncertainNumber.UP.local_optimisation import local_optimisation_method 
import numpy as np
from PyUncertainNumber.UP.uncertaintyPropagation import up_bb

In [ ]:
from PyUncertainNumber import UncertainNumber as UN
from PyUncertainNumber.UP.vertex import vertexMethod
# from PyUncertainNumber.UP.performance_func import cantilever_beam_func, cantilever_beam_deflection, cantilever_beam_stress

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext rich

# Problem statement

The numerical example models a simple cantilever beam with length, $L$, distance to the neutral axis $y$, Young’s modulus, $E$, second moment of inertia, $I$, and external load, $F$. We will compute the bending stress $σ$, and the deflection $d$, assuming the above input parameters are **uncertain numbers**.

We will use this example throughout the document to illustrate the differences among various uncertainty quantification approaches. A complete description of the following methods can be found in the [DAWS1 report](https://sites.google.com/view/dawsreports/up).

![alt text](../assets/cantilever.png)

**Fig.1** Cantilever beam with input parameters.  

# Create the propagating function

A propagating function is created which calculates the stress, $σ$, as a function of $L$, $y$, $E$, $I$, and $F$.
- The function's input is only the uncertain numbers.
- As sampling techniques are used it is likely that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this.


Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.

In [3]:
def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:
        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
        deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    except:
        deflection = np.nan

    return deflection

In [ ]:
# know the function with default Python help mechanism
cantilever_beam_deflection?

In [ ]:
from PyUncertainNumber.UP.performance_func import cantilever_beam_deflection, cantilever_beam_stress

## Verify the function

To ensure that the function yields meaningful results. We consider that input has the nominal values seen below.

The function should yield deflection equal to 0.162m.
 

In [ ]:
# test the function

y  = 0.155 # m
L  = 10.05 # m
I = 0.000386 # m**4
F = 37 # kN
E = 200 # GPa

x = np.array([L, I, F, E])
deflection = cantilever_beam_deflection(x)

print(deflection) # 0.162m

# Propagate epistemic uncertainty 



## Uncertainty Characterisation

Construct `UncertainNumbers` objects for the inputs assuming they are intervals with the lower and upper values are seen below

- $y = [0.145, 0.155] \ m$

- $L = [9.95, 10.05] \ m$

- $I = [0.0003861591, 0.0005213425] \ m^{4}$

- $F = [11, 37] \ kN$

- $E = [200, 220] \ GPa$
 

In [ ]:
y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155])
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds=[9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds=[0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds=[11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])

## Uncertainty Propagation methods 

Choose from a suite of black box propagating techniques to propagate the intervals through the model. 

### Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of $n = 2^{d}$. 

For the working example, there are d = 5 intervals which results in $n = 2^{5} = 32$ input combinations.

#### Assumptions




- [x] Ioanna's original top-level UP function with  the 'endspoints' method

In [4]:
L =  np.array([9.95, 10.05]) # m

I =  np.array([0.0003861591, 0.0005213425])# m**4

F =  np.array([11, 37]) # kN

E =  np.array([200, 220]) # GPa

# Create a 2D np.array with all uncertain input parameters in the **correct** order.
xInt = np.array([L, I, F, E])
print(xInt)

''' naive imple of hint on combination number of endpoints method '''
# # How many input combinations are expected from the endpoint propagation?
# d = xInt.shape[0] # is the number of uncertain input expressed as intervals.
# n = 2**d # The total number of combinations 
# print("Total number of input combinations for the endpoint method:", n) 

[[9.950000e+00 1.005000e+01]
 [3.861591e-04 5.213425e-04]
 [1.100000e+01 3.700000e+01]
 [2.000000e+02 2.200000e+02]]


In [5]:
''' original from Ioanna '''

# method = "endpoint"
# base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
# min_y, max_y, x_miny, x_maxy = main.up_bb(xInt, cantilever_beam_deflection, n = None, method= method, save_raw_data = "no", base_path = base_path)

# #TODO envelope the results of min_y nad max_y 
# # plot the extremities for each input variable
# print(min_y)
# print(max_y)


# The endpoint method is being called 

METHOD = "endpoint"
base_path = ""

# TODO return either as namedTuple or dict to be explicit
a = up_bb(xInt, 
          cantilever_beam_deflection, 
          n = None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = base_path)
              

#min_y, max_y, x_miny, x_maxy = up.endpoints_method(xInt, cantilever_beam_deflection, save_raw_data = 'no')

# # check the results.
# # print(min_y)
# # print(max_y)
# # print(x_miny)
# # print(x_maxy)

Total number of input combinations for the endpoint method: 16


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 24556.81it/s]

There are no NA values produced by the input


- [x] Leslie's standalone endspoints implementation

In [ ]:
deflection_bounds_e = UN.endpointsMethod(vars=['L', 'I', 'F', 'E'], 
                                         func=cantilever_beam_deflection, 
                                         name='deflection', 
                                         symbol='D')
deflection_bounds_e

In [ ]:
stress_bounds_e = UN.endpointsMethod(vars=['y', 'L', 'I', 'F'], 
                                     func=cantilever_beam_stress, 
                                     name='stress', 
                                     symbol='S')
stress_bounds_e

### Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [ ]:
''' naive imple of hint on combination number of endpoints method '''
# How many input combinations are expected from the subinterval reconstitution propagation?
d = 5 # is the number of uncertain input expressed as intervals.
m = 4 # is the number of partitions. 
n = (m+1)**d # The total number of combinations 
print("Total number of input combinations for the subinterval reconstitution method:", n) 

<font color='red'>subinterval reconstitution method not implemented ?</font>

### Sampling approach

- Brute Monte Carlo
- Latin Hypercube

In [7]:
METHOD = "montecarlo"
base_path = ""
a = up_bb(xInt, 
          cantilever_beam_deflection, 
          n = 10, 
          method = METHOD, 
          save_raw_data="yes", 
          base_path = base_path)
              
#min_y, max_y, x_miny, x_maxy = up.endpoints_method(xInt, cantilever_beam_deflection, save_raw_data = 'no')

# check the results.
# print(min_y)
# print(max_y)
# print(x_miny)
# print(x_maxy)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 14271.73it/s]


In [ ]:
# Check the results

### Optimisation approach

- Local optimisation
- Genetic algorithm


In [ ]:
METHOD = "local_optimisation"
#base_path = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\cantilever_beam"
a = main.up_bb(xInt, cantilever_beam_deflection, x0 = None, method = method, method_loc = 'Nelder-Mead')
              
#print(a)

In [ ]:
# The local_optimisation is being called 
a = local_optimisation_method(xInt, cantilever_beam_deflection, x0 = None, method_loc = 'Nelder-Mead')

# check the results.
print(a)

### Naive interval arithmetics

In [ ]:
defl = cantilever_beam_deflection(L, I, F, E)

In [ ]:
defl

# Mixed types of uncertainty

when inputs have various types of uncertainty.

In [ ]:
#TODO could Scott elaborate on envelop?